In [23]:
import os
import cv2
import numpy as np

In [24]:
browser_name = "LetterNone"
output_training_folder = r"c:\Users\tamas\LettersTraining\None"
output_testing_folder = r"c:\Users\tamas\LettersTesting\None"
image_name = "Video_Video"
name_prefix = "letters_"

test_split_ratio = 0.05
shuffle_test = False

if not os.path.exists(output_training_folder):
    os.makedirs(output_training_folder)
    print("Created folder: " + output_training_folder)

if not os.path.exists(output_testing_folder):
    os.makedirs(output_testing_folder)
    print("Created folder: " + output_testing_folder)

Created folder: c:\Users\tamas\LettersTraining\None
Created folder: c:\Users\tamas\LettersTesting\None


In [25]:
# Make sure input sequence browser and video exists

browser_node = slicer.util.getFirstNodeByName(browser_name, className='vtkMRMLSequenceBrowserNode')
if browser_node is None:
    logging.error("Could not find browser node: {}".format(browser_name))
    raise

image_node = slicer.util.getFirstNodeByName(image_name)
if image_node is None:
    logging.error("Could not find image node: {}".format(image_name))
    raise

n = browser_node.GetNumberOfItems()

print("Number of images in browser: {}".format(n))

Number of images in browser: 887


In [26]:
# Prepare test image indices

n_test = int(n * test_split_ratio)
all_indices = range(n)
if shuffle_test:
    test_indices = np.random.choice(all_indices, n_test)
else:
    test_indices = range(n_test)

browser_node.SelectFirstItem()

for i in range(n):
    image = image_node.GetImageData()
    shape = list(image.GetDimensions())
    shape.reverse()
    components = image.GetNumberOfScalarComponents()
    if components > 1:
        shape.append(components)
        shape.remove(1)
    image_array = vtk.util.numpy_support.vtk_to_numpy(image.GetPointData().GetScalars()).reshape(shape)
    image_name = name_prefix + "%04d" % i + ".png"
    if i in test_indices:
        image_fullname = os.path.join(output_testing_folder, image_name)
    else:
        image_fullname = os.path.join(output_training_folder, image_name)
    image_brg = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_fullname, image_brg)
    browser_node.SelectNextItem()
    slicer.app.processEvents()

print("Exported {} image in {}".format(n_test, output_testing_folder))
print("Exported {} image in {}".format((n-n_test), output_training_folder))

Exported 44 image in c:\Users\tamas\LettersTesting\None
Exported 843 image in c:\Users\tamas\LettersTraining\None
